# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/swetharangu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/swetharangu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/swetharangu/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [2]:
import pandas as pd
import numpy as np
import os
import pickle
from sqlalchemy import create_engine
import re
import nltk

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier,AdaBoostClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, accuracy_score, f1_score, fbeta_score, classification_report

from scipy.stats import hmean
from scipy.stats.mstats import gmean

In [3]:
# load data from database
engine = create_engine('sqlite:///disaster_resp_db.db')
df = pd.read_sql_table('disaster_resp_table', engine)

df.describe()

,id,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
count,26216.00000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,...,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000
mean,15224.82133,0.773650,0.170659,0.004501,0.414251,0.079493,0.050084,0.027617,0.017966,0.032804,...,0.011787,0.043904,0.278341,0.082202,0.093187,0.010757,0.093645,0.020217,0.052487,0.193584
std,8826.88914,0.435276,0.376218,0.066940,0.492602,0.270513,0.218122,0.163875,0.132831,0.178128,...,0.107927,0.204887,0.448191,0.274677,0.290700,0.103158,0.291340,0.140743,0.223011,0.395114
min,2.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,7446.75000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,15662.50000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,22924.25000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,30265.00000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [4]:
# Given value 2 in the related field are neglible so it could be error. Replacing 2 with 1 to consider it a valid response.
# Alternatively, we could have assumed it to be 0 also. In the absence of information I have gone with majority class.
df['related']=df['related'].map(lambda x: 1 if x == 2 else x)

In [5]:
X = df['message']
y = df.iloc[:,4:]

### 2. Write a tokenization function to process your text data

In [6]:
def tokenize(text):
    """
    Tokenize the text function
    
    INPUT 
        text: Text to be processed   
    OUTPUT
        Returns a processed text variable that was tokenized, lower cased, stripped, and lemmatize
    """
    
    # Extract the word tokens from the provided text
    tokens = nltk.word_tokenize(text)
    
    #Lemmanitizer to remove inflectional and derivationally related forms of a word
    lemmatizer = nltk.WordNetLemmatizer()

    # List of clean tokens
    clean_tokens = [lemmatizer.lemmatize(w).lower().strip() for w in tokens]
    return clean_tokens

In [ ]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

In [ ]:
def multioutput_fscore(y_true,y_pred,beta=1):
    """
    To create table from classification_report output and calculate F1 score & accuracy mean by providing beta=1 value 
    
    INPUT 
        Y_true and y_pred values   
    OUTPUT
        Returns calculated mean value for both F1-score and accuracy.
    """
    
    score_list = []
    if isinstance(y_pred, pd.DataFrame) == True:
        y_pred = y_pred.values
    if isinstance(y_true, pd.DataFrame) == True:
        y_true = y_true.values
    for column in range(0,y_true.shape[1]):
        score = fbeta_score(y_true[:,column],y_pred[:,column],beta,average='weighted')
        score_list.append(score)
    f1score_numpy = np.asarray(score_list)
    f1score_numpy = f1score_numpy[f1score_numpy<1]
    f1score = gmean(f1score_numpy)
    return  f1score

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [7]:
pipeline = Pipeline(
                   [('vect', CountVectorizer(tokenizer = tokenize)),
                    ('tfidf', TfidfTransformer()),
                    ('clf', MultiOutputClassifier(RandomForestClassifier()))
                    ])

#pipeline_new for testing better parameters
#Pipeline to add new features #StartingVerbExtractor to the dataset and applying 
#Adaboost classifier for better classification techniques 

pipeline_new = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),

            ('starting_verb', StartingVerbExtractor())
        ])),

        ('clf', MultiOutputClassifier(AdaBoostClassifier()))
    ])


### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y)
pipeline.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x7fd17d2838b0>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

In [9]:
y_pred_test = pipeline.predict(X_test)
y_pred_train = pipeline.predict(X_train)

In [16]:
msg = ['need water urgently']
test_output = pipeline.predict(msg)

print(y_train.columns.values[(test_output.flatten()==1)])

['related' 'aid_related' 'water']


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [28]:
y_pred_pd = pd.DataFrame(y_pred_test, columns = y_test.columns)

for column in y_test.columns:
    print('------------------------------------------------------\n')
    print('FEATURE: {}\n'.format(column))
    print(classification_report(y_test[column],y_pred_pd[column]))

------------------------------------------------------

FEATURE: related

              precision    recall  f1-score   support

           0       0.74      0.26      0.39      1506
           1       0.82      0.97      0.89      5048

    accuracy                           0.81      6554
   macro avg       0.78      0.62      0.64      6554
weighted avg       0.80      0.81      0.77      6554

------------------------------------------------------

FEATURE: request

              precision    recall  f1-score   support

           0       0.89      0.99      0.94      5416
           1       0.89      0.42      0.57      1138

    accuracy                           0.89      6554
   macro avg       0.89      0.70      0.75      6554
weighted avg       0.89      0.89      0.87      6554

------------------------------------------------------

FEATURE: offer

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6524
           1     

/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.96      1.00      0.98      6280
           1       0.86      0.09      0.17       274

    accuracy                           0.96      6554
   macro avg       0.91      0.55      0.57      6554
weighted avg       0.96      0.96      0.95      6554

------------------------------------------------------

FEATURE: other_aid

              precision    recall  f1-score   support

           0       0.87      1.00      0.93      5697
           1       0.60      0.02      0.03       857

    accuracy                           0.87      6554
   macro avg       0.74      0.51      0.48      6554
weighted avg       0.84      0.87      0.81      6554

------------------------------------------------------

FEATURE: infrastructure_related

              precision    recall  f1-score   support

           0       0.94      1.00      0.97      6169
           1       0.00      0.00      0.00       385

    accuracy     

In [29]:
#get calculated values using multioutput_fscore function

multi_f1 = multioutput_fscore(y_test,y_pred_test, beta = 1)

multi_f1_train = multioutput_fscore(y_train,y_pred_train, beta = 1)

overall_accuracy = (y_pred_test == y_test).mean().mean()

print('Average overall accuracy {0:.2f}% \n'.format(overall_accuracy*100))
print('F1 score (custom definition) {0:.2f}%\n'.format(multi_f1*100))

#metrics.classification_report(y_test, y_pred_test, target_names=y.columns.values)

/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:68: FutureWarning: Pass beta=1 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


Average overall accuracy 94.57% 

F1 score (custom definition) 92.74%



### 6. Improve your model
Use grid search to find better parameters. 

In [31]:
pipeline_new.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'features', 'clf', 'features__n_jobs', 'features__transformer_list', 'features__transformer_weights', 'features__verbose', 'features__text_pipeline', 'features__starting_verb', 'features__text_pipeline__memory', 'features__text_pipeline__steps', 'features__text_pipeline__verbose', 'features__text_pipeline__vect', 'features__text_pipeline__tfidf', 'features__text_pipeline__vect__analyzer', 'features__text_pipeline__vect__binary', 'features__text_pipeline__vect__decode_error', 'features__text_pipeline__vect__dtype', 'features__text_pipeline__vect__encoding', 'features__text_pipeline__vect__input', 'features__text_pipeline__vect__lowercase', 'features__text_pipeline__vect__max_df', 'features__text_pipeline__vect__max_features', 'features__text_pipeline__vect__min_df', 'features__text_pipeline__vect__ngram_range', 'features__text_pipeline__vect__preprocessor', 'features__text_pipeline__vect__stop_words', 'features__text_pipeline__vect__strip_accents

In [20]:
#Trying to retreive best parameters to Adaboost using GridSearchCV to get better F1Score and Accuracy than Random classifier.

parameters = {
#        'clf__estimator__learning_rate': [0.01, 0.02, 0.05]
         'clf__estimator__n_estimators': [10, 20]
        ,'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
#       'features__text_pipeline__vect__max_df': (0.75, 1.0),
#       'features__text_pipeline__vect__max_features': (None, 5000),
#       'features__text_pipeline__tfidf__use_idf': (True, False)
        }

cv = GridSearchCV(estimator= pipeline_new, param_grid= parameters, scoring='f1_micro',verbose=2, n_jobs=-1)

cv.fit(X_train, y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 out of  20 | elapsed:  6.0min remaining:  1.5min
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:  7.0min finished


GridSearchCV(estimator=Pipeline(steps=[('features',
                                        FeatureUnion(transformer_list=[('text_pipeline',
                                                                        Pipeline(steps=[('vect',
                                                                                         CountVectorizer(tokenizer=<function tokenize at 0x7fd17d2838b0>)),
                                                                                        ('tfidf',
                                                                                         TfidfTransformer())])),
                                                                       ('starting_verb',
                                                                        StartingVerbExtractor())])),
                                       ('clf',
                                        MultiOutputClassifier(estimator=AdaBoostClassifier()))]),
             n_jobs=-1,
             param_grid={'clf__estim

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  


Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [21]:
# Get the prediction values from the gridsearchCV

y_prediction_test = cv.predict(X_test)
y_prediction_train = cv.predict(X_train)

In [35]:
msg = ['hungry and sick']
test_output = cv.predict(msg)

print(y_train.columns.values[(test_output.flatten()==1)])

['related' 'aid_related']


In [32]:
#calling function with values to get claculated F1 score and accuracy.

GCV_multi_f1 = multioutput_fscore(y_test,y_prediction_test, beta = 1)

GCV_multi_f1_train = multioutput_fscore(y_train,y_prediction_train, beta = 1)

GCV_overall_accuracy = (y_prediction_test == y_test).mean().mean()

print('Average overall accuracy {0:.2f}% \n'.format(GCV_overall_accuracy*100))
print('F1 score (custom definition) {0:.2f}%\n'.format(GCV_multi_f1*100))

#print('F1 score (train) {0:.2f}%\n'.format(multi_f1_train*100))

Average overall accuracy 94.71% 

F1 score (custom definition) 93.53%



### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [ ]:
m = pickle.dumps('classifier.pkl')

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.